In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Dropout, LSTM
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn
from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import accuracy_score


In [2]:
features = ['Time', 'L1' , 'L2', 'L3', 'L4', 'L5', 'L6', 'L7', 'L8', 
            'R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8', 
            'Total_Force_Left', 'Total_Force_Right']

In [3]:
def load_file(filepath):
	dataframe = pd.read_csv(filepath, header=None, delim_whitespace=True)
	return dataframe.values

# load a list of files into a 3D array of [samples, timesteps, features]
def load_category(filenames, prefix=''):
	loaded = list()
	for name in filenames:
		data = load_file(prefix + name)
		loaded.append(data)
	# stack group so that features are the 3rd dimension
	loaded = np.dstack(loaded)
	return loaded


# load a dataset category, such as train or test
def load_dataset_category(category, prefix=''):
	# load all 19 files as a single array
	filenames = []
	for fname in features:
		filenames.append(category + '/' + fname + '_' + category + '.txt')

	X = load_category(filenames, prefix)
	y = load_file(prefix + 'y_'+ category +'.txt')
	hy = load_file(prefix + 'hyscore_'+ category +'.txt')
	return X, y, hy

# returns train and test X and y elements
def load_dataset(prefix=''):
	trainX, trainy, trainHY = load_dataset_category('train', prefix)
	print(trainX.shape, trainy.shape, trainHY.shape)
	testX, testy, testHY = load_dataset_category('test', prefix)
	print(testX.shape, testy.shape, testHY.shape)
 
	# zero-offset class values
	trainy = trainy - 1
	testy = testy - 1
	# one hot encode
	trainy = tf.keras.utils.to_categorical(trainy)
	trainHY = tf.keras.utils.to_categorical(trainHY)
	testy = tf.keras.utils.to_categorical(testy)
	testHY = tf.keras.utils.to_categorical(testHY)  
 
	return trainX, trainy, trainHY, testX, testy, testHY

In [4]:
trainX, trainy, trainHY, testX, testy, testHY = load_dataset('Final/')
print("Training Data = ", trainX.shape)
print("Training Class = ", trainy.shape)
print("Training HoehnYahr Class = ", trainHY.shape)
print("Test Data = ", testX.shape)
print("Test Class = ", testy.shape)
print("Test HoehnYahr Class = ", testHY.shape)

(30104, 100, 19) (30104, 1) (30104, 1)
(13761, 100, 19) (13761, 1) (13761, 1)
Training Data =  (30104, 100, 19)
Training Class =  (30104, 2)
Training HoehnYahr Class =  (30104, 4)
Test Data =  (13761, 100, 19)
Test Class =  (13761, 2)
Test HoehnYahr Class =  (13761, 3)


In [5]:
def build_lstm_model(input_data, output_size, neurons, activ_func='linear',
                     dropout=0.2, loss='mse', optimizer='adam'):
    model = Sequential()
    model.add(LSTM(neurons, input_shape=(input_data.shape[1], input_data.shape[2])))
    model.add(Dropout(dropout))
    model.add(Dense(units=output_size))
    model.add(Activation(activ_func))
    model.compile(loss=loss, optimizer=optimizer)
    model.summary()
    return model
np.random.seed(245)
window_len = 5
test_size = 0.2
zero_base = True
lstm_neurons = 50
epochs = 20
batch_size = 32
loss = 'mse'
dropout = 0.24
optimizer = 'adam'

In [6]:
model = build_lstm_model(
    trainX, output_size=1, neurons=lstm_neurons, dropout=dropout, loss=loss,
    optimizer=optimizer)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50)                14000     
                                                                 
 dropout (Dropout)           (None, 50)                0         
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
 activation (Activation)     (None, 1)                 0         
                                                                 
Total params: 14,051
Trainable params: 14,051
Non-trainable params: 0
_________________________________________________________________


In [7]:
phy_history = model.fit(trainX,
                        trainy,
                        validation_data=(testX, testy),
                        epochs = epochs,
                        batch_size = batch_size,
                        verbose = 1,
                        shuffle=True,
                        )

Epoch 1/20
941/941 [==============================] - 49s 49ms/step - loss: 0.3026 - val_loss: 0.2510
Epoch 2/20
941/941 [==============================] - 46s 48ms/step - loss: 0.2532 - val_loss: 0.2502
Epoch 3/20
941/941 [==============================] - 44s 46ms/step - loss: 0.2508 - val_loss: 0.2501
Epoch 4/20
941/941 [==============================] - 45s 48ms/step - loss: 0.2501 - val_loss: 0.2500
Epoch 5/20
941/941 [==============================] - 44s 47ms/step - loss: 0.2500 - val_loss: 0.2500
Epoch 6/20
941/941 [==============================] - 45s 48ms/step - loss: 0.2500 - val_loss: 0.2500
Epoch 7/20
941/941 [==============================] - 45s 48ms/step - loss: 0.2500 - val_loss: 0.2500
Epoch 8/20
941/941 [==============================] - 45s 48ms/step - loss: 0.2500 - val_loss: 0.2500
Epoch 9/20
941/941 [==============================] - 45s 48ms/step - loss: 0.2500 - val_loss: 0.2500
Epoch 10/20
941/941 [==============================] - 44s 47ms/step - loss: 0.250

In [8]:
def model_evaluate(model, target, testX, testy):
  y_pred = model.predict(testX)
  y_pred = np.argmax(y_pred, axis=1)
  y_test = np.argmax(testy, axis=1)

  cm = tf.math.confusion_matrix(labels = y_test, predictions = y_pred)
  # Calculate accuracy
  cm_np = cm.numpy()
  conf_acc = (cm_np[0, 0] + cm_np[1, 1])/ np.sum(cm_np) * 100
  print("Accuracy for Test Data = ", conf_acc)

In [9]:
model_evaluate(model, [0,1], testX, testy)

Accuracy for Test Data =  66.07077973984448
